In [ ]:
EXAMPLE_1 = "../example_1.txt"
EXAMPLE_2 = "../example_2.txt"
INPUT = "../input.txt"

In [ ]:
def parse_input(input_file_name):
    wires = []
    gates = []
    with open(input_file_name, 'r') as f:
        reading_wires = True
        for line in f:
            if line == '\n':
                reading_wires = False
                continue
            if reading_wires:
                wire, value = line.strip().replace('\n', '').split(':')
                value = bool(int(value.strip()))
                wires.append((wire, value))
            else:
                inputs, output = line.strip().replace('\n', '').split('->')
                output = output.strip()
                input0, gate, input1 = [s.strip() for s in inputs.strip().split(' ')]
                gates.append((input0, gate, input1, output))
    return wires, gates

In [ ]:
wires, gates = parse_input(EXAMPLE_1)
print(wires, gates)

For part 1, we use a Gate structure that stores all the gates that depend on the output of each one. 

This way, when one gate has both input value set, its output can be generated and all the gates that depend on it can be updated in turn.

A wire is simply a special "PASS" gate that has two identical inputs and whose output is equal to the input value.

In [ ]:
class Gate:
    op: str
    inputs: list[bool | None]
    output: bool | None
    children: list[str]

    def __init__(self, op):
        self.output = None
        self.inputs = [None, None]
        self.op = op
        self.children = []

    def add_child(self, name):
        self.children.append(name)

    def set_input(self, value, device):
        for i in range(2):
            if self.inputs[i] is None:
                self.inputs[i] = value
                break
        if None not in self.inputs:
            self.generate_output(device)

    def generate_output(self, device):
        if self.inputs[0] is None or self.inputs[1] is None:
            return
        match self.op:
            case "AND":
                self.output = self.inputs[0] and self.inputs[1]
            case "OR":
                self.output = self.inputs[0] or self.inputs[1]
            case "XOR":
                self.output = self.inputs[0] ^ self.inputs[1]
            case "PASS":
                self.output = self.inputs[0]
        for child in self.children:
            device[child].set_input(self.output, device)

The device is simply the collection of all the gates, with their connections to each other.

The name of a gate is the name of its output.

In [ ]:
def build_device(wires, gates):
    DEVICE = {}
    for name, _ in wires:
        new_gate = Gate('PASS')
        DEVICE[name] = new_gate

    for input0, op, input1, name in gates:
        # Inputs and outputs are not initialized in the same way
        # It all depends on which side of a gate a name first appears in the list
        if name in DEVICE:
            # The current output already appeared as an input
            DEVICE[name].op = op
        else:
            # The current output has never been seen before
            new_gate = Gate(op=op)
            DEVICE[name] = new_gate
        for input in [input0, input1]:
            if input in DEVICE:
                # The current input already appeared as an input or an output
                DEVICE[input].add_child(name)
            else:
                # The current input has never been seen before
                DEVICE[input] = Gate(op=None)
                DEVICE[input].add_child(name)
    return DEVICE

In [ ]:
DEVICE = build_device(wires, gates)

To start the device, we simply initialize the inputs for the wire gates. Both inputs need to be set in order for the gate output to be generated. We could also have a special case in the Gate class definition for wires but it doesn't really make a difference.

In [ ]:
def start_device(device, wires):
    for name, state in wires:
        device[name].set_input(state, device)
        device[name].set_input(state, device)

In [ ]:
start_device(DEVICE, wires)

Once the device is started, it runs until all gates have received their two inputs and generated their output.

In [ ]:
for name, gate in DEVICE.items():
    print(name, gate.output)

Now we just need a function to calculate the binary value on the $z_i$ gates

In [ ]:
def get_number(device):
    number = 0
    for name, gate in device.items():
        if name.startswith('z'):
            index = int(name[1:])
            value = int(gate.output)
            number += 2**index * value
    return number

In [ ]:
print(get_number(DEVICE))

In [ ]:
def part_1(input_file_name):
    wires, gates = parse_input(input_file_name)
    DEVICE = build_device(wires, gates)
    start_device(DEVICE, wires)
    result = get_number(DEVICE)
    print(result)

In [ ]:
part_1(EXAMPLE_1)

In [ ]:
part_1(EXAMPLE_2)

In [ ]:
part_1(INPUT)

The device is supposed to be an adder. Let's define how an adder actually works, bit by bit, and then we'll need to compare the device we have with the theoretical definitions in order to find discrepancies.

For bit 0 we simply have:

$z_{00} = x_{00} \land y_{00}$

Starting at bit 1 we have to take into account the value that carries over ($c$) from the previous bit.

$z_{01} = (x_{01} \land y_{01}) \land c_{00}$

with $c_{00} = x_{00} \mathrel{\&} y_{00}$

And the value to carry over from bit 1 to bit 2 is:

$c_{01} = (x_{01} \mathrel{\&} y_{01}) \mid ((x_{01} \land y_{01}) \mathrel{\&} c_{00})$

So in general we have:

$z_{i} = (x_{i} \land y_{i}) \land c_{i-1}$

$c_{i} = (x_{i} \mathrel{\&} y_{i}) \mid ((x_{i} \land y_{i}) \mathrel{\&} c_{i-1})$

In [ ]:
wires, gates = parse_input(INPUT)
nb_of_bits = len(wires) // 2

From now on we'll call the wires registers because we'll treat them as such.

So first we need functions to find how registers are defined in our device

In [ ]:
def get_output_regs_for_inputs(input0, input1, gates):
    result = {}
    for (i0, op, i1, output) in gates:
        # The order of the inputs is random so we sort them to order the inputs
        if sorted([input0, input1]) == sorted([i0, i1]):
            # We store the output register based on the operation needed to get it from the inputs
            result[op] = output
    return result

def get_input_regs_for_output(output, gates):
    result = {}
    for (i0, op, i1, o) in gates:
        if o == output:
            # We order the inputs and store them based on the operation needed to get the output from them
            result[op] = sorted([i0, i1])
    return result

As we go through all the bits, we'll encounter some discrepancies and we'll need to swap registers.

In [ ]:
def swap_regs(gates, regs_to_swap):
    # Update the gate definitions whose output is one of the registers to swap
    for k, (i0, op, i1, output) in enumerate(gates):
        swapped_output = None
        for idx, g in enumerate(regs_to_swap):
            # We need to check if the output reg has already been swapped, so we don't swap it back to its original value
            if output == g and swapped_output is None:
                swapped_output = regs_to_swap[1-idx]
        gates[k] = (i0, op, i1, swapped_output if swapped_output is not None else output)

In [ ]:
def check_bit(bit_nb, prev_c, gates, debug):
    x_i = f"x{bit_nb:02d}"
    y_i = f"y{bit_nb:02d}"
    z_i = f"z{bit_nb:02d}"

    # Look for the gates that calculate xi ^ yi and xi & yi
    output_regs = get_output_regs_for_inputs(x_i, y_i, gates)
    # Store the registers that hold each value
    xi_xor_yi = output_regs["XOR"]
    xi_and_yi = output_regs["AND"]
    # xi | yi is never needed so it shouldn't be calculated
    if "OR" in output_regs:
        print(f"Error: gate calculating {x_i} | {y_i}")
        raise(ValueError)

    if debug:
        print(f"{x_i} AND {y_i} = {xi_and_yi}")
        print(f"{x_i} XOR {y_i} = {xi_xor_yi}")
        print(f"Prev c = {prev_c}")
        
    # Bit 0 is a special case because there is no previous value to carry over
    if bit_nb == 0:
        # z_00 should be equal to x0 ^ y0 so we just need to check that
        if xi_xor_yi == z_i:
            # z00 is correctly defined
            if debug:
                print(f"z00 correctly defined: {y_i} XOR {x_i} = {z_i}")
        else:
            print(f"z00 incorrectly defined")
            raise(ValueError)
        # The value to carry over is x00 & y00
        c = xi_and_yi
        # There are no registers to swap
        return True, c, []

    # General case: bit_nb >= 1

    # Look for the gates that calculate (xi ^ yi) ^ ci-1 and (xi ^ yi) & ci-1
    output_regs = get_output_regs_for_inputs(xi_xor_yi, prev_c, gates)
    # The or value is not needed so it shouldn't be calculated
    if "OR" in output_regs:
        print(f"Error: gate calculating (xi ^ yi) | ci-1")
        raise(ValueError)
    if "XOR" in output_regs:
        # The correct gate to calculate z_i exists, we need to check that its output is actually z_i
        should_be_z_i = output_regs["XOR"]
        if should_be_z_i == z_i:
            # z_i is correctly defined
            if debug:
                print(f"{z_i} correctly defined: {xi_xor_yi} XOR {prev_c} = {z_i}")
        else:
            # The register holding (xi ^ yi) ^ ci-1 is not equal to z_i so we need to swap it with z_i
            if debug:
                print(f"Need to swap {z_i} with {should_be_z_i}")
            regs_to_swap = [z_i, should_be_z_i]
            return False, None, regs_to_swap
    else:
        # The correct gate to calculate z_i doesn't exist, meaning either xi ^ yi or ci-1 are not set to the correct register
        # Look for the gate that defines z_i
        input_regs = get_input_regs_for_output(z_i, gates)
        # We should find a XOR gate only
        if "AND" in input_regs or "OR" in input_regs:
            print(f"Error: AND or OR gate resulting in {z_i}")
            raise(ValueError)
        if "XOR" in input_regs:
            swap = False
            if xi_xor_yi == input_regs["XOR"][0]:
                if debug:
                    print(f"Need to swap {input_regs["XOR"][1]} with {prev_c}")
                regs_to_swap = [prev_c, input_regs["XOR"][1]]
                swap = True
            elif xi_xor_yi == input_regs["XOR"][1]:
                if debug:
                    print(f"Need to swap {input_regs["XOR"][0]} with {prev_c}")
                regs_to_swap = [prev_c, input_regs["XOR"][0]]
                swap = True
            elif prev_c == input_regs["XOR"][0]:
                if debug:
                    print(f"Need to swap {input_regs["XOR"][1]} with {xi_xor_yi}")
                regs_to_swap = [xi_xor_yi, input_regs["XOR"][1]]
                swap = True
            elif prev_c == input_regs["XOR"][1]:
                if debug:
                    print(f"Need to swap {input_regs["XOR"][0]} with {xi_xor_yi}")
                regs_to_swap = [xi_xor_yi, input_regs["XOR"][0]]
                swap = True
            if swap:
                return False, None, regs_to_swap
            else:
                print(f"Error: couldn't find swap to fix {z_i}")
                raise(ValueError)
        else:
            print(f"Error: no XOR gate resulting in {z_i}")
            raise(ValueError)


    if "AND" in output_regs:
        # The correct gate to calculate the right hand part of c_i exists, we need to check that its output is actually used with xi & yi to get the c_i
        right_reg_for_c = output_regs["AND"]
        output_regs = get_output_regs_for_inputs(xi_and_yi, right_reg_for_c, gates)
        if "OR" in output_regs:
            c = output_regs["OR"]
            return True, c, []
        else:
            print(f"Missing OR gate for {xi_and_yi} and {right_reg_for_c}")
            raise(ValueError)
    else:
        print(f"Error: no AND gate resulting in right hand of c_{nb_of_bits}")
        raise(ValueError)

In [ ]:
def find_regs_to_swap(gates, nb_of_bits, debug=False):
    regs_to_swap = []
    bit_nb = 0
    c = None
    while bit_nb < nb_of_bits:
        bit_is_valid, new_c, new_regs_to_swap = check_bit(bit_nb, c, gates, debug)
        if new_c is not None:
            c = new_c
        if bit_is_valid:
            bit_nb += 1
        else:
            regs_to_swap.extend(new_regs_to_swap)
            swap_regs(gates, new_regs_to_swap)
    return regs_to_swap

In [ ]:
find_regs_to_swap(gates, nb_of_bits, debug=True)

In [ ]:
def part_1(input_file_name):
    wires, gates = parse_input(input_file_name)
    nb_of_bits = len(wires) // 2
    regs_to_swap = find_regs_to_swap(gates, nb_of_bits)
    print(",".join(sorted(regs_to_swap)))

In [ ]:
part_1(INPUT)